In [1]:
pip install git+https://github.com/KinWaiCheuk/nnAudio.git#subdirectory=Installation

  Cloning https://github.com/KinWaiCheuk/nnAudio.git to /tmp/pip-req-build-e4ig8864
  Running command git clone --filter=blob:none --quiet https://github.com/KinWaiCheuk/nnAudio.git /tmp/pip-req-build-e4ig8864
  Resolved https://github.com/KinWaiCheuk/nnAudio.git to commit ef941aff84a0c1a97456093044e8d318b5f49ade
  Preparing metadata (setup.py) ... done
  Created wheel for nnAudio: filename=nnAudio-0.3.2-py3-none-any.whl size=43594 sha256=01e88d9711f6ac97063e3b5be9511f49014452e9040b606feb2f7ab9c3f3f98a
  Stored in directory: /tmp/pip-ephem-wheel-cache-gecasybw/wheels/1d/a5/c7/323cb6cb45785232ac4ec085a130ec5a5642be599143dd5a17
Successfully built nnAudio
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install einops

  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


https://github.com/lucidrains/vit-pytorch/blob/main/vit_pytorch/vit.py
https://github.com/YuanGongND/ast/blob/master/src/models/ast_models.py
https://arxiv.org/pdf/2104.01778.pdf
https://arxiv.org/pdf/2010.11929.pdf

In [6]:
import torch
import torch.nn as nn
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from nnAudio import features
import numpy as np
import math
from einops import rearrange, reduce, asnumpy, parse_shape
from einops.layers.torch import Rearrange, Reduce
from timm.models.vision_transformer import Mlp, PatchEmbed , _cfg
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from torchmetrics.classification import Accuracy
from tqdm.notebook import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
# sample_audio, sr = torchaudio.load('/kaggle/input/audioset/train_wav/-0DLPzsiXXE.wav')

In [8]:
# sample_audio = sample_audio.mean(0)[:sr * 10].unsqueeze(0)

In [9]:
# sample_audio.shape

In [10]:
# sr

In [11]:
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

In [12]:
256*3

768

In [13]:
class PatchEmbedding(nn.Module):
    """
    Slight modification of PatchEmbeding introduced in ViT
    in_channels = 3 in the original paper but here spectrogram (the image) is 1-dimensional
    """
    def __init__(self, fstride=10, tstride=10, patch_size = 16, emb_size = 768):
        super().__init__()
        self.patch_size = patch_size
        self.projection = nn.Conv2d(in_channels = 1, out_channels = emb_size, 
                                    kernel_size = (patch_size, patch_size), stride = (fstride,tstride))
    def gen_maskid_frame(self, num_patches_to_mask=3):
        mask_id = random.sample(range(0, self.num_patches), num_patches_to_mask)
        return mask_id
    
    def forward(self, x):
        x = self.projection(x).flatten(2).transpose(1, 2)
        return x

In [14]:
class Attention(nn.Module):
    # taken from https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        q = q * self.scale

        attn = (q @ k.transpose(-2, -1))
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [15]:
class Block(nn.Module):
    # taken from https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm,Attention_block = Attention,Mlp_block=Mlp
                 ,init_values=1e-4):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention_block(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp_block(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x 

In [16]:
class MEASTModel(nn.Module):
    # Adapted from https://github.com/YuanGongND/ast
    """
    The Memory-Efficient Audio Spectrogram Tranformer Model.
    """
    def __init__(self, depth = 12, audio_length = 10, label_dim=527, sr = 44100, fstride=10, tstride=10, input_fdim=128, num_heads = 8, embedding_dim = 768):
        super().__init__()
        self.input_tdim = fstride * tstride * audio_length
        f_dim, t_dim = self.get_shape(fstride, tstride, input_fdim, self.input_tdim)
        num_patches = f_dim * t_dim
        self.num_patches = num_patches
        self.embedding_dim = embedding_dim
        self.spec_layer = features.MelSpectrogram(n_mels = input_fdim, win_length = 25, 
                                             hop_length = sr // (fstride * tstride) - 4, 
                                             center=False, sr=sr, pad_mode = "same", 
                                             trainable_mel = True)
        self.patch_embedding = PatchEmbedding(fstride=fstride, tstride=tstride)
        self.pos_embedding = nn.Parameter(torch.zeros(1, self.num_patches + 2, self.embedding_dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embedding_dim))
        self.dist_token = nn.Parameter(torch.randn(1, 1, self.embedding_dim))
        self.blocks = nn.ModuleList([Block(dim=self.embedding_dim, num_heads=12) for i in range(depth)])
        self.pos_drop = nn.Dropout()
        self.dropout = nn.Dropout()
        self.norm = nn.LayerNorm(self.embedding_dim) #if not use_fc_norm else nn.Identity()
        self.mlp_head = nn.Sequential(nn.LayerNorm(self.embedding_dim), nn.Linear(self.embedding_dim, label_dim))
        
    def get_shape(self, fstride, tstride, input_fdim=128, input_tdim=1024):
        test_input = torch.randn(1, input_fdim, input_tdim)
        test_proj = nn.Conv2d(1, 3* (16 ** 3), kernel_size=(16, 16), stride=(fstride, tstride))
        test_out = test_proj(test_input)
        f_dim = test_out.shape[1]
        t_dim = test_out.shape[2]
        return f_dim, t_dim

    def forward(self, x):
        """
        :param x: the input spectrogram, expected shape: (batch_size, time_frame_num, frequency_bins), e.g., (12, 1024, 128)
        :return: prediction
        """
        # expect input x = (batch_size, time_frame_num, frequency_bins), e.g., (12, 1024, 128)
        #x = x.unsqueeze(1)
        B = x.shape[0]
        #print(x.shape) torch.Size([12, 1, 220500])
        x = self.spec_layer(x)
        #print(x.shape) torch.Size([12, 128, 500])
        x = x.mT.unsqueeze(1)
        #print(x.shape) torch.Size([12, 1, 500, 128])
        x = self.patch_embedding(x)
        # print(x.shape) torch.Size([12, 588, 768])
        #x = x.unsqueeze(0) #.flatten(2) #.mT
        # print(x.shape) #torch.Size([1, 451584, 12])
        cls_tokens = self.cls_token.expand(B, -1, -1)
        dist_token = self.dist_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, dist_token, x), dim=1)
        x = x + self.pos_embedding
        x = self.pos_drop(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        x = torch.mean(x, dim = 1)
        x = self.mlp_head(x)

        return x


https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py#L494


In [17]:
batch_size = 12
device = 'cuda:0'

In [18]:
class AudioSet(Dataset):
    def __init__(self, root_dir, filenames, transform=None):
        self.root_dir = root_dir
        self.filenames = filenames
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        file_name = self.filenames[idx]
        label = file_name.split('-')[-1].split('.')[0]
        file_path = f'{self.root_dir}/{file_name}'
        waveform, sample_rate = torchaudio.load(file_path)
        if self.transform:
            sample = self.transform(input_freq = sample_rate)(waveform)
        return {'audio': sample, 'label' : torch.tensor(int(label), dtype = torch.float32)}

In [19]:
len(os.listdir('/kaggle/input/environmental-sound-classification-50/audio/audio/'))

2002

In [20]:
class AudioProcessing(torch.nn.Module):
    def __init__(
        self,
        input_freq = None,
        resample_freq=44100
    ):
        super().__init__()
        self.resample = T.Resample(orig_freq=input_freq, new_freq=resample_freq)
        self.sr = input_freq

    def forward(self, waveform):
        waveform = waveform.mean(0)[:self.sr * 5].unsqueeze(0)
        resampled = self.resample(waveform)
        return resampled

In [21]:
df = pd.read_csv("/kaggle/input/environmental-sound-classification-50/esc50.csv")

In [22]:
df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [23]:
msk = np.random.rand(len(df)) < 0.8
train_df_pre = df[msk].reset_index(drop=True)
test_df = df[~msk]

In [24]:
train_df = train_df_pre.sample(frac=0.8,random_state=200).reset_index(drop=True)
valid_df = train_df_pre.drop(train_df.index).reset_index(drop=True)

In [25]:
root_dir = '/kaggle/input/environmental-sound-classification-50/audio/audio/44100'

In [26]:
train_dataset = AudioSet(root_dir = root_dir,
                         filenames = train_df['filename'],
                         transform = AudioProcessing)

In [27]:
valid_dataset = AudioSet(root_dir = root_dir,
                        filenames = valid_df['filename'],
                        transform = AudioProcessing)

In [28]:
test_dataset = AudioSet(root_dir = root_dir,
                        filenames = test_df['filename'],
                        transform = AudioProcessing)

In [29]:
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [30]:
train_dl

In [31]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

1310
328
362


In [32]:
x = next(iter(train_dl))['audio']

In [33]:
model = MEASTModel(label_dim = 50, audio_length = 5).to(device)

STFT kernels created, time used = 0.0979 seconds
STFT filter created, time used = 0.0045 seconds
Mel filter created, time used = 0.0045 seconds


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # use smaller lr
accuracy = Accuracy(task="multiclass", num_classes=50).to(device)
num_epochs = 100

for epoch in range(num_epochs):    
    # Get the next batch from seq_dl
    train_loss = 0.0
    train_acc = 0.0
    total = 0
    for batch in tqdm(train_dl):
        batch['label'] = batch['label'].type(torch.LongTensor) 
        batch_x = batch['audio'].to(device)
        batch_y = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += accuracy(outputs, batch_y) * len(batch_x)
        total += len(batch_x)

    print(f'Epoch {epoch} loss: {train_loss:.4f} train_acc : {train_acc/total:.4f}')
            
    valid_acc = 0.0
    total = 0
    with torch.no_grad():
        for batch in tqdm(valid_dl):
            batch['label'] = batch['label'].type(torch.LongTensor) 
            batch_x = batch['audio'].to(device)
            batch_y = batch['label'].to(device)
            outputs = model(batch_x)
            
            valid_acc += accuracy(outputs, batch_y) * len(batch_x)
            total += len(batch_x)
            
        print(f"valid_acc: {valid_acc/total:.4f}")

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch 0 loss: 438.8759 train_acc : 0.0374


  0%|          | 0/28 [00:00<?, ?it/s]

valid_acc: 0.0518


  0%|          | 0/110 [00:00<?, ?it/s]

...

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch 19 loss: 88.9324 train_acc : 0.7504


  0%|          | 0/28 [00:00<?, ?it/s]

valid_acc: 0.7012


  0%|          | 0/110 [00:00<?, ?it/s]